In [1]:
# !wget https://huggingface.co/datasets/mesolitica/translate-evol-instruct-indonesian/resolve/main/translate-evol-instruct-indonesian.jsonl

In [2]:
roles = {
    'user': '<manusia>',
    'assistant': '<bot>'
}

system = [
    'always reply in indon',
    'sentiasa respond dalam bahasa indonesia',
    'sentiasa jawab in indon',
    'awak adalah chatbot yang respond dalam bahasa indonesia',
    'you are a chatbot that always respond in indonesian',
    'u always reply in indonesian',
    'awak sentiasa reply in bahasa indon',
    'you are a chatbot that always reply in indon',
    'you are an assistant that always reply in indon',
    'awak adalah chatbot yang sentiasa respond dalam indon',
    'awak adalah pembantu AI yang reply in indonesian',
    'reply in indon',
    'Anda adalah chatbot yang merespon dalam bahasa Indonesia',
    'Anda adalah chatbot yang merespon dalam indon',
]

In [3]:
import json
import random

In [4]:
instructions = []

with open('translate-evol-instruct-indonesian.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        
        a = []
        
        if 'translation' in l['conversations'][1]['value'].lower():
            continue
        
        if 'answer' in l['conversations'][1]['value'].lower():
            continue
            
        if 'question' in l['conversations'][1]['value'].lower():
            continue
            
        if 'translate' in l['conversations'][1]['value'].lower():
            continue
        
        s = random.choice(system)
        t = l['conversations'][0]['value_ms']
        instruct = []
        if random.random() > 0.5:
            prompt_input = s
            q = t
        else:
            prompt_input = None
            q = f'{s}\n\n{t}'
        
        t = l['conversations'][1]['value']
            
        d = {
            'prompt_input': prompt_input,
            'input': q,
            'output': t,
        }

        instructions.append(d)
            
len(instructions)   

43204

In [5]:
instructions[-2]

{'prompt_input': 'reply in indon',
 'input': "Siapakah watak Shylock dalam karya 'The Merchant of Venice' karya Shakespeare?",
 'output': 'Shylock adalah karakter antagonis dalam karya "The Merchant of Venice" karya William Shakespeare. Dia adalah seorang Yahudi kaya yang merupakan seorang pemilik toko dan pengumpul utang di Venesia pada abad ke-16. Shylock umumnya digambarkan sebagai sosok yang kejam dan tidak empati, karena dia ingin memberlakukan perjanjian yang sangat keras pada karakter utama, Antonio.\n\nShylock memegang dendam yang besar terhadap Antonio karena merasa bahwa dia telah merendahkan dan menghina agama Yahudi. Ketika Antonio meminta pinjaman dari Shylock untuk membiayai bisnisnya, Shylock bersedia memberikan pinjaman dengan syarat jika Antonio tidak dapat membayar kembali pada waktu yang ditentukan, maka Shylock berhak memotong satu pon daging dari tubuh Antonio sesuai dengan syarat kontrak.\n\nShylock digambarkan sebagai sosok yang rakus atas uang dan tidak kenal be

In [6]:
answers = []
with open('translate-evol-instruct-indonesian.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        
        if 'translation' in l['conversations'][1]['value'].lower():
            continue
        
        if 'answer' in l['conversations'][1]['value'].lower():
            continue
            
        if 'question' in l['conversations'][1]['value'].lower():
            continue
            
        if 'translate' in l['conversations'][1]['value'].lower():
            continue
        
        l_left = len(l['conversations'][1]['value'].split())
        l_right = len(l['conversations'][1]['value_ms'].split())
        
        if l_right / l_left < 0.95:
            continue
        
        l['conversations'][1]['value'], l['conversations'][1]['value_ms']
        answers.append((l['conversations'][1]['value'], l['conversations'][1]['value_ms']))
len(answers)

39100

In [7]:
answers = sorted(answers, key = lambda x: len(x[0]), reverse = True)

In [8]:
template = [
    'teks: {teks}\nterjemah ke standard malay',
    'teks: {teks}\nterjemah ke melayu baku',
    '`{teks}` terjemah ke malay',
    'terjemah ke bahasa melayu `{teks}`',
]

In [9]:
for a in answers:
    left = random.choice(template).format(teks = a[0])
    d = {
        'prompt_input': None,
        'input': left,
        'output': a[1],
    }
    instructions.append(d)

In [10]:
template = [
    'teks: {teks}\nterjemah ke standard indonesian',
    'teks: {teks}\nterjemah ke bahasa indon',
    '`{teks}` terjemah ke indon',
    'terjemah ke bahasa indon `{teks}`',
]

In [11]:
for a in answers:
    left = random.choice(template).format(teks = a[1])
    d = {
        'prompt_input': None,
        'input': left,
        'output': a[0],
    }
    instructions.append(d)

In [12]:
len(instructions)

121404

In [13]:
import pandas as pd

pd.DataFrame(instructions).to_parquet('prepared-force-indonesian.parquet')

In [ ]:
from huggingface_hub import HfApi
api = HfApi()

api.upload_file(
    path_or_fileobj="prepared-force-indonesian.parquet",
    path_in_repo='data/force_indonesian-00000-of-00001.parquet',
    repo_id="mesolitica/Malaysian-SFT",
    repo_type="dataset",
)

prepared-force-indonesian.parquet:   0%|          | 0.00/189M [00:00<?, ?B/s]